In [2]:
import cv2
import numpy as np
import time
import threading
import math
import random
from collections import Counter
from IPython.display import clear_output

import sys, os
sys.path.append("../x11-window-interactor")
from x11_interactor import X11WindowInteractor

In [3]:
# Initialize mouse and keyboard controllers
interactor = X11WindowInteractor()

# Global variable to control the start and stop of the script
script_running = False

# Main Roi
main_roi = {'left': 0, 'top': 0, 'width': 3840, 'height': 2160}

# HOH Presurge (ROI)
hoh_presurge_roi = {'left': 1600, 'top': 200, 'width': 200, 'height': 200}

# Dark Portal postsurge
dark_portal_postsurge = {'left':1650, 'top':815, 'width':60, 'height':60}

Click on the target window after running this...


In [4]:
def find_image(template_path, screenshot):
    # Load the template image and convert to grayscale
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    template_edges = cv2.Canny(template, 0, 100)

    # Convert screenshot to grayscale
    gray = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2GRAY)
    screenshot_edges = cv2.Canny(gray, 0, 100)

    # Perform template matching using the edge images
    result = cv2.matchTemplate(screenshot_edges, template_edges, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)
    # print(max_val)
    # Get the coordinates and dimensions of the matched region
    h, w = template.shape
    top_left = max_loc
    return top_left[0], top_left[1], w, h

In [5]:
def randomize_click_position(center_x, center_y, width, height, shape='rectangle', roi_diminish=2):
    if shape == 'circle':
        # Randomize within a circle
        radius = (min(width, height) // 2) // roi_diminish
        angle = np.random.uniform(0, 2 * math.pi)
        r = radius * np.sqrt(np.random.uniform(0, 1))
        click_x = int(center_x + r * math.cos(angle))
        click_y = int(center_y + r * math.sin(angle))
    else:
        # Randomize within a rectangle
        area_width = width * 0.5
        area_height = height * 0.5

        left_bound = center_x - area_width / roi_diminish
        right_bound = center_x + area_width / roi_diminish
        top_bound = center_y - area_height / roi_diminish
        bottom_bound = center_y + area_height / roi_diminish

        click_x = int(np.random.normal(center_x, area_width / 6))
        click_y = int(np.random.normal(center_y, area_height / 6))

        click_x = min(max(click_x, left_bound), right_bound)
        click_y = min(max(click_y, top_bound), bottom_bound)

    return int(click_x), int(click_y)

In [ ]:
def main_script():
    bank_boat_img = 'bank_boat.png'
    load_last_bank_preset_img = 'load_last_preset.png'
    passing_bracelet_img = 'passing_bracelet.png'
    rub_passing_bracelet_img = 'rub_passing_bracelet.png'
    haunt_on_the_hill_img = 'haunt_on_the_hill.png'
    dark_portal_presurge_img = 'dark_portal_presurge.png'
    flesh_altar_presurge_img = 'flesh_altar_presurge.png'
    flesh_altar_postsurge_img = 'flesh_altar_postsurge.png'
    miasma_altar_presurge_img = 'miasma_altar_presurge.png'
    miasma_altar_postsurge_img = 'miasma_altar_postsurge.png'
    
    global script_running

    cycle_count = int(input("Enter the number of runecrafting runs: "))
    rune_type = input("Enter the type of rune to craft (miasma/flesh): ")

    # Capture the initial screen and do calibration

    # Bank boat
    interactor.activate()
    screenshot = interactor.capture()
    bank_boat_data = find_image(bank_boat_img, screenshot)
    if not bank_boat_data:
        print("Bank boat not found.")
        return
    x, y, w, h = bank_boat_data
    center_x, center_y = x + w // 2, y + h // 2
    bank_boat_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
    click_position(bank_boat_pos, speed=0.2 ,right_click=True)
    time.sleep(random.uniform(0.6, 0.7))

    # Load last preset
    screenshot = capture_screen()
    load_last_bank_preset_data = find_image(load_last_bank_preset_img, screenshot)
    x, y, w, h = load_last_bank_preset_data
    center_x, center_y = x + w // 2, y + h // 2
    load_last_bank_preset_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
    click_position(load_last_bank_preset_pos, speed=0.2 ,right_click=False)
    time.sleep(random.uniform(0.6, 0.7))

    # Passing Bracelet
    screenshot = capture_screen()
    passing_bracelet_data = find_image(passing_bracelet_img, screenshot)
    if not passing_bracelet_data:
        print("Passing bracelet not found.")
        return
    x, y, w, h = passing_bracelet_data
    center_x, center_y = x + w // 2, y + h // 2
    passing_bracelet_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
    click_position(passing_bracelet_pos, speed=0.2 ,right_click=True)
    time.sleep(random.uniform(0.6, 0.7))

    # Rub Passing bracelet
    screenshot = capture_screen()
    rub_passing_bracelet_data = find_image(rub_passing_bracelet_img, screenshot)
    x, y, w, h = rub_passing_bracelet_data
    center_x, center_y = x + w // 2, y + h // 2
    h = h//3
    rub_passing_bracelet_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
    click_position(rub_passing_bracelet_pos, speed=0.2 ,right_click=False)
    time.sleep(random.uniform(0.7, 0.9))

    # Haunt on the hill teleport
    screenshot = capture_screen()
    haunt_on_the_hill_data = find_image(haunt_on_the_hill_img, screenshot)
    if haunt_on_the_hill_data:
        press_key('2')
        press_key('2')
    time.sleep(random.uniform(5.5, 6))

    # Presurge Darkportal location
    screenshot = capture_roi(hoh_presurge_roi)
    dark_portal_presurge_data = find_image(dark_portal_presurge_img, screenshot)
    x, y, w, h = dark_portal_presurge_data
    x += hoh_presurge_roi['left']
    y += hoh_presurge_roi['top']
    center_x, center_y = x + w // 2, y + h // 2
    dark_portal_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='circle', roi_diminish=3)
    click_position(dark_portal_presurge_pos, speed=0.2 ,right_click=False)
    time.sleep(random.uniform(1.4, 1.6))
    surge()
    time.sleep(random.uniform(0.65, 0.7))

    # Postsurge Darkportal location
    dark_portal_postsurge_data = dark_portal_postsurge.values()
    x, y, w, h = dark_portal_postsurge_data
    center_x, center_y = x + w // 2, y + h // 2
    dark_portal_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='circle', roi_diminish=3)
    click_position(dark_portal_postsurge_pos, speed=0.2 ,right_click=False)
    time.sleep(random.uniform(1.2, 1.3))

    if rune_type=='miasma':
        # Find Miasma altar and surge
        screenshot = capture_screen()
        miasma_altar_presurge_data = find_image(miasma_altar_presurge_img, screenshot)
        x, y, w, h = miasma_altar_presurge_data
        center_x, center_y = x + w // 2, y + h // 2
        miasma_altar_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        click_position(miasma_altar_presurge_pos, speed=0.2 ,right_click=False)
        time.sleep(random.uniform(0.6, 0.65))
        surge()
        time.sleep(random.uniform(1.15, 1.25))

        # Re-search Miasma altar and craft runes
        screenshot = capture_screen()
        miasma_altar_postsurge_data = find_image(miasma_altar_postsurge_img, screenshot)
        x, y, w, h = miasma_altar_postsurge_data
        center_x, center_y = x + w // 2, y + h // 2
        miasma_altar_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        press_key('s')
        click_position(miasma_altar_postsurge_pos, speed=0.2 ,right_click=False)
        time.sleep(random.uniform(1.15, 1.25))

    if rune_type=='flesh':
        # Find Flesh altar and surge
        screenshot = capture_screen()
        flesh_altar_presurge_data = find_image(flesh_altar_presurge_img, screenshot)
        x, y, w, h = flesh_altar_presurge_data
        w = w//2
        center_x, center_y = x + w // 2, y + h // 2
        flesh_altar_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        click_position(flesh_altar_presurge_pos, speed=0.2 ,right_click=False)
        time.sleep(random.uniform(0.6, 0.65))
        surge()
        time.sleep(random.uniform(1.15, 1.25))

        # Re-search Flesh altar and craft runes
        screenshot = capture_screen()
        flesh_altar_postsurge_data = find_image(flesh_altar_postsurge_img, screenshot)
        x, y, w, h = flesh_altar_postsurge_data
        center_x, center_y = x + w // 2, y + h // 2
        flesh_altar_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        press_key('s')
        click_position(flesh_altar_postsurge_pos, speed=0.2 ,right_click=False)
        time.sleep(random.uniform(1.6, 1.8))

    for _ in range(np.random.randint(4,6)):
        press_key('a')
    time.sleep(random.uniform(3.9, 4.1))

    iteration = 0

    while script_running:
        if not script_running:
            break
        # Step 1 -> Right click bank boat
        x, y, w, h = bank_boat_data
        center_x, center_y = x + w // 2, y + h // 2
        bank_boat_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        click_position(bank_boat_pos, speed=0.2 ,right_click=True)
        time.sleep(random.uniform(0.55, 0.65))
        
        if not script_running:
            break
        # Step 2 -> Find and left-click on "Load last bank preset"
        screenshot = capture_screen()
        load_last_bank_preset_data = find_image(load_last_bank_preset_img, screenshot)
        x, y, w, h = load_last_bank_preset_data
        center_x, center_y = x + w // 2, y + h // 2
        load_last_bank_preset_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        click_position(load_last_bank_preset_pos, speed=0.2 ,right_click=False)
        time.sleep(random.uniform(0.55, 0.65))
        
        if not script_running:
            break
        # Step 3 -> Right click the passing bracelet
        x, y, w, h = passing_bracelet_data
        center_x, center_y = x + w // 2, y + h // 2
        passing_bracelet_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        click_position(passing_bracelet_pos, speed=0.2 ,right_click=True)
        time.sleep(random.uniform(0.55, 0.65))
        
        if not script_running:
            break
        # Step 4 -> Rub Passing bracelet
        screenshot = capture_screen()
        rub_passing_bracelet_data = find_image(rub_passing_bracelet_img, screenshot)
        x, y, w, h = rub_passing_bracelet_data
        center_x, center_y = x + w // 2, y + h // 2
        h = h//3
        rub_passing_bracelet_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        click_position(rub_passing_bracelet_pos, speed=0.2 ,right_click=False)
        time.sleep(random.uniform(0.5, 0.6))

        if not script_running:
            break
        # Step 5 -> Haunt on the hill teleport
        screenshot = capture_screen()
        haunt_on_the_hill_data = find_image(haunt_on_the_hill_img, screenshot)
        if haunt_on_the_hill_data:
            press_key('2')
            press_key('2')
        time.sleep(random.uniform(2.8, 3.0))

        if not script_running:
            break
        # Step 6 -> Surge to Dark Portal
        x, y, w, h = dark_portal_presurge_data
        x += hoh_presurge_roi['left']
        y += hoh_presurge_roi['top']
        center_x, center_y = x + w // 2, y + h // 2
        dark_portal_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='circle', roi_diminish=3)
        click_position(dark_portal_presurge_pos, speed=0.2 ,right_click=False)
        time.sleep(random.uniform(1.4, 1.6))
        surge()
        time.sleep(random.uniform(0.65, 0.7))

        if not script_running:
            break
        # Step 6 -> Enter the Dark Portal
        x, y, w, h = dark_portal_postsurge_data
        center_x, center_y = x + w // 2, y + h // 2
        dark_portal_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='circle', roi_diminish=3)
        click_position(dark_portal_postsurge_pos, speed=0.2 ,right_click=False)
        time.sleep(random.uniform(1.2, 1.3))

        if not script_running:
            break
        if rune_type=='miasma':
            # Step 7 -> Find Miasma altar and surge
            x, y, w, h = miasma_altar_presurge_data
            center_x, center_y = x + w // 2, y + h // 2
            miasma_altar_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
            click_position(miasma_altar_presurge_pos, speed=0.2 ,right_click=False)
            time.sleep(random.uniform(0.6, 0.65))
            surge()
            time.sleep(random.uniform(1.15, 1.25))

            if not script_running:
                break
            # Step 8 -> Re-search Miasma altar and craft runes
            x, y, w, h = miasma_altar_postsurge_data
            center_x, center_y = x + w // 2, y + h // 2
            miasma_altar_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
            press_key('s')
            click_position(miasma_altar_postsurge_pos, speed=0.2 ,right_click=False)
            time.sleep(random.uniform(1.15, 1.25))

        if rune_type=='flesh':
            # Step 7 -> Find Flesh altar and surge
            x, y, w, h = flesh_altar_presurge_data
            w = w//2
            center_x, center_y = x + w // 2, y + h // 2
            flesh_altar_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
            click_position(flesh_altar_presurge_pos, speed=0.2 ,right_click=False)
            time.sleep(random.uniform(0.6, 0.65))
            surge()
            time.sleep(random.uniform(1.15, 1.25))

            if not script_running:
                break
            # Step 8 -> Re-search flesh altar and craft runes
            x, y, w, h = flesh_altar_postsurge_data
            center_x, center_y = x + w // 2, y + h // 2
            flesh_altar_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
            press_key('s')
            click_position(flesh_altar_postsurge_pos, speed=0.2 ,right_click=False)
            time.sleep(random.uniform(1.6, 1.8))
            

        for _ in range(np.random.randint(4,6)):
            press_key('a')
        time.sleep(random.uniform(3.9, 4.1))
        
        if not script_running:
            break
        clear_output(wait=True)
        print(f"Crafted Iteration: {iteration}")
        iteration+=1
        if iteration == cycle_count:
            print("Cycles Completed")
            script_running = not script_running

In [ ]:
def on_press(key):
    global script_running
    if key == pkeyboard.KeyCode(char='-'):
        script_running = not script_running
        if script_running:
            print("Script started.")
            # Run the main script in a separate thread
            threading.Thread(target=main_script).start()
        else:
            print("Script stopped.")

In [ ]:
def start_listener():
    with pkeyboard.Listener(on_press=on_press) as listener:
        listener.join()

In [14]:
bank_boat_img = 'assets/bank_boat.png'
load_last_bank_preset_img = 'assets/load_last_preset.png'
passing_bracelet_img = 'assets/passing_bracelet.png'
rub_passing_bracelet_img = 'assets/rub_passing_bracelet.png'
haunt_on_the_hill_img = 'assets/haunt_on_the_hill.png'
dark_portal_presurge_img = 'assets/dark_portal_presurge.png'
flesh_altar_presurge_img = 'assets/flesh_altar_presurge.png'
flesh_altar_postsurge_img = 'assets/flesh_altar_postsurge.png'
miasma_altar_presurge_img = 'assets/miasma_altar_presurge.png'
miasma_altar_postsurge_img = 'assets/miasma_altar_postsurge.png'

global script_running

# Capture the initial screen and do calibration

# Bank boat
interactor.activate()
screenshot = interactor.capture()
bank_boat_data = find_image(bank_boat_img, screenshot)
if not bank_boat_data:
    print("Bank boat not found.")
else:  # Only draw the rectangle if the boat is found
    x, y, w, h = bank_boat_data
    center_x, center_y = x + w // 2, y + h // 2
    bank_boat_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)

    # Draw the rectangle on the screenshot
    top_left = (x, y)
    bottom_right = (x + w, y + h)
    color = (0, 255, 0)  # Green rectangle (BGR format)
    thickness = 2
    cv2.rectangle(screenshot, top_left, bottom_right, color, thickness)

    cv2.namedWindow("Bank Boat Detection", cv2.WINDOW_NORMAL)
    cv2.imshow("Bank Boat Detection", screenshot)
    cv2.waitKey(0)  # Wait for any key press
    cv2.destroyAllWindows()


    interactor.click(bank_boat_pos[0], bank_boat_pos[1], 3)

In [ ]:
cv2.namedWindow("test", cv2.WINDOW_NORMAL)
interactor.activate()
while True:
    img = interactor.capture()
    cv2.imshow("test", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Interrupted by user.")
        break
cv2.destroyAllWindows()